# RD-Detector

### Import statement 

In [ ]:
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from nltk.corpus import wordnet
from nltk.corpus import stopwords


import pandas as pd
import numpy as np

### data path configuration

#all_discussions.csv format: 
#id_discussion;category_type;author;date;#comments;title

Example:

36047;help;ammund0;2022-07-02T22:08:29Z;1;[svg's with gatsby] - gatsby build process removing inline styles from svg's
36639;help;gallardox;2022-09-19T17:42:09Z;1;how to deploy the production gatsby site.
36608;rfc;haledgarbaya;2022-09-14T12:41:41Z;5;rfc: partial hydration
36609;umbrella-discussions;haledgarbaya;2022-09-14T12:43:02Z;5;gatsby 5 umbrella discussion
36635;help;amccall0;2022-09-18T23:15:24Z;0;can't deploy gatsbywp to netlify or gatsby cloud
36063;help;rTab;2022-07-05T15:36:05Z;1;control gatsby router behaviour
36633;help;ilyntalmers;2022-09-18T19:12:26Z;0;critical css extraction
36631;help;aushalyap;2022-09-18T11:06:46Z;0;can we expect at least dev server performance improvement in v5?
36626;help;tephyswe;2022-09-16T16:33:40Z;2;gatsby build service w/o restoring cache?

In [ ]:
#project_name = "homebrew"
#repo = "Homebrew/discussions/"
#f1 = "all_discussions.csv"

project_name = "gatsby"
repo = "gatsbyjs/gatsby/"
f1 = "all_discussions.csv"

#repoprojeto = "vercel/next.js/"
#project_name = "next_js"
#f1 = "all_discussions.csv"

path_data = "github/"+project_name+"/"

### RD-Detector: preprocessing

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def apply_lemmatization(sentence):
    
    texto = " ".join([wordnet_lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence) if w not in string.punctuation])
    return(texto)

In [ ]:
new_stop = "e.g,0o,0s,3a,3b,3d,6b,6o,a,a1,a2,a3,a4,ab,able,about,above,abst,ac,ad,ae,af,ag,ah,ain,ain't,aj,al,all,am,among,amongst,amoungst,an,and,announce,another,any,ao,ap,ar,are,aren,arent,aren't,as,a's,aside,at,au,auth,av,aw,away,awfully,ax,ay,az,b,b1,b2,b3,ba,back,bc,bd,be,bi,bill,biol,bj,bk,bl,bn,both,bottom,bp,br,brief,briefly,bs,bt,bu,but,bx,by,c,c1,c2,c3,ca,call,came,can,cannot,cant,can't,cause,causes,cc,cd,ce,certainly,cf,cg,ch,ci,cit,cj,cl,clearly,cm,c'mon,cn,co,com,come,comes,con,couldn,couldnt,couldn't,cp,cq,cr,cry,cs,c's,ct,cu,currently,cv,cx,cy,cz,d,d2,da,date,dc,dd,de,df,di,did,didn,didn't,dj,dk,dl,do,doe,does,doesn,doesn't,doing,don,done,don't,down,downwards,dp,dr,ds,dt,du,due,during,dx,dy,e,e2,e3,ea,each,ec,ed,edu,ee,ef,effect,eg,ei,ej,el,eleven,else,elsewhere,em,en,end,ending,eo,ep,eq,er,es,especially,est,et,et-al,etc,eu,ev,even,ever,every,everybody,everyone,everything,everywhere,ex,ey,f,f2,fa,far,fc,few,ff,fi,fifteen,fifth,fify,fill,find,fire,first,five,fix,fj,fl,fn,fo,former,fr,from,front,fs,ft,fu,full,fy,g,ga,gave,ge,get,gets,getting,gi,give,given,gives,giving,gj,gl,go,goes,going,gone,got,gotten,gr,greetings,gs,gy,h,h2,h3,had,hadn,hadn't,happens,hardly,has,hasn,hasnt,hasn't,have,haven,haven't,having,he,hed,he'd,he'll,hello,help,hence,her,here,hereafter,hereby,herein,heres,here's,hereupon,hers,herself,hes,he's,hh,hi,hid,him,himself,his,hither,hj,ho,home,hopefully,how,howbeit,however,how's,hr,hs,http,hu,hundred,hy,i,i2,i3,i4,i6,i7,i8,ia,ib,ibid,ic,id,i'd,ie,if,ig,ignored,ih,ii,ij,il,i'll,im,i'm,in,inasmuch,inc,indeed,insofar,instead,interest,invention,inward,io,ip,iq,ir,is,isn,isn't,it,itd,it'd,it'll,its,it's,itself,iv,i've,ix,iy,iz,j,jj,jr,js,jt,ju,just,k,ke,keep,keeps,kept,kg,kj,km,know,known,knows,ko,l,l2,la,largely,last,lately,later,latter,latterly,lb,lc,le,least,les,less,lest,let,lets,let's,lf,like,liked,likely,lj,ll,ll,ln,lo,look,looking,looks,los,lr,ls,lt,ltd,m,m2,ma,made,mainly,make,makes,many,may,maybe,me,mean,means,meantime,meanwhile,merely,mg,might,mightn,mightn't,mill,million,mine,miss,ml,mn,mo,more,moreover,most,mostly,move,mr,mrs,ms,mt,mu,much,mug,must,mustn,mustn't,my,myself,n,n2,na,name,namely,nay,nc,nd,ne,near,nearly,necessarily,necessary,need,needn,needn't,needs,neither,never,nevertheless,new,next,ng,ni,nine,ninety,nj,nl,nn,no,nobody,non,none,nonetheless,noone,nor,normally,nos,not,noted,nothing,now,nowhere,nr,ns,nt,ny,o,oa,ob,oc,od,of,off,often,og,oh,oi,oj,ok,okay,ol,old,om,omitted,on,once,one,ones,only,onto,oo,op,oq,or,ord,os,ot,other,others,otherwise,ou,ought,our,ours,ourselves,out,outside,over,overall,ow,owing,own,ox,oz,p,p1,p2,p3,par,part,pas,past,pc,pd,pe,people,per,perhaps,pf,ph,pi,pj,pk,pl,please,plus,pm,pn,po,poorly,pp,pq,pr,previously,primarily,probably,promptly,proud,provides,ps,pt,pu,put,py,q,qj,qu,que,quickly,quite,qv,r,r2,ra,ran,rather,rc,rd,re,readily,really,rf,rh,ri,right,rj,rl,rm,rn,ro,rq,rr,rs,rt,ru,run,rv,ry,s,s2,sa,said,same,saw,say,saying,says,sc,sd,se,sec,second,secondly,section,see,seeing,seem,seemed,seeming,seems,seen,self,selves,sensible,sent,serious,seriously,seven,several,sf,shall,shan,shan't,she,shed,she'd,she'll,shes,she's,should,shouldn,shouldn't,should've,show,showed,shown,showns,shows,si,sj,sl,slightly,sm,sn,so,some,somebody,somehow,someone,somethan,something,sometime,sometimes,somewhat,somewhere,soon,sorry,sp,sq,sr,ss,st,still,strongly,sub,substantially,sufficiently,sup,sure,sy,system,sz,t,t1,t2,t3,tb,tc,td,te,tell,ten,tends,tf,th,than,thank,thanks,thanx,that,that'll,thats,that's,that've,the,their,theirs,them,themselves,then,thence,there,thereafter,thereby,thered,therefore,therein,there'll,thereof,therere,theres,there's,thereto,thereupon,there've,these,they,theyd,they'd,they'll,theyre,they're,they've,thickv,thin,think,third,this,thorough,thoroughly,those,thou,though,thoughh,throug,through,throughout,thru,thus,ti,til,tip,tj,tl,tm,tn,to,too,took,top,toward,towards,tp,tq,tr,truly,ts,t's,tt,tx,u,u201d,ue,ui,uj,uk,um,un,unfortunately,unless,unlike,unlikely,until,unto,uo,ur,us,ut,v,va,vd,ve,ve,very,via,viz,vj,vo,vol,vols,volumtype,vq,vs,vt,vu,w,wa,want,wants,was,wasn,wasnt,wasn't,way,we,wed,we'd,welcome,we'll,well-b,went,were,we're,weren,werent,weren't,we've,what,whatever,what'll,whats,what's,when,whence,whenever,when's,whereafter,whereas,whereby,wherein,wheres,where's,whereupon,whim,whither,who,whod,whoever,whole,who'll,whom,whomever,whos,who's,whose,why,why's,wi,widely,will,willing,wish,wo,won,wonder,wont,won't,words,world,would,wouldn,wouldnt,wouldn't,www,xf,xi,xj,xk,xl,xn,xo,xs,xt,xv,xx,y,y2,yes,yet,yj,yl,you,youd,you'd,you'll,your,youre,you're,yours,yourself,yourselves,you've,yr,ys,yt,z,zero,zi,zz"
l_new_stop = new_stop.split(",") 

stop_words1 = set(stopwords.words('english'))
stop_words1.update(l_new_stop)

In [ ]:
def remove_stopword(t):
    word_tokens = word_tokenize(t)
    filtered_sentence = " "
    for w in word_tokens: 
        if w not in stop_words1: 
            filtered_sentence += " " + w
    
    return(str(filtered_sentence))
    

In [ ]:
def remove_especific_dot(t):
    word_tokens = word_tokenize(t)
    filtered_sentence = " "
    for w in word_tokens: 
        if "." in w: 
            dados = w.split(".")
            if (len(dados[0])==0 or len(dados[1])==0):
                w = w.replace("."," ")
        filtered_sentence += " " + w

    
    return(str(filtered_sentence))

In [ ]:
def is_number(sentence):
    
    texto = " ".join(w for w in nltk.word_tokenize(sentence) if not w.isnumeric())
    return(texto)

In [ ]:
def remove_url(t):
    return re.sub(r"http\S+", " ", t)

In [ ]:
def pre_process(text):

    punctuations = '''!`()[]{};|:'"\/,<>?=@$%^&*~├ ┌ ● ’ ▲↵·“”'''
    
    # lowercase
    text=text.lower()

    #remove url
    text = remove_url(text)
    
    text = text.replace("c:\\","")
      
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
       
    #remove emoji   
    emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       u"\U0001f926-\U0001f937"
                       u"\u200d"
                       u"\u2640-\u2642" 
                       "]+", flags=re.UNICODE)
    
    text = emoji_pattern.sub(u'', text)
   

    #remove dots
    no_punct = " "
    for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
        else:
            no_punct = no_punct + " "
    text=no_punct
    
    text = remove_especific_dot(text)
  
      
    #remove number   
    text = is_number(text)
    
       
    text = apply_lemmatization(text)
    
    #remove stopword
    text = remove_stopword(text)
    
    return text.strip()

In [ ]:
def read_file_lines(path,f):
    f = open(path+f,'r', encoding='utf-8')
    lines = f.readlines()
    return (lines)

In [ ]:
def read_file_content(path,f):
    l = read_file_lines(path,f)
    text= ""
    text = " ".join(l)
    return (text)

In [ ]:
def save_tupla(l_tuplas, f, path=path_data):
    
    file = open(path+f,"a")
    for l in l_tuplas:
        file.write(str(l[0])+";"+str(l[1])+";"+str(l[2])+"\n")
    file.close()
    

In [ ]:
def save_file(path,t,name_file):
    file = open(path+name_file,"a")
    file.write(t+'\n')
    file.close()

### Loading and preprocessing discussion posts

In [ ]:
dir_disc_main_post = "clean_semComment_3"


#Set the catecory filter c
category_types = ['help', 'ideas', 'ideas-feature-requests', 'umbrella-discussions', 'community', 'rfc','ALL']
category = category_types[2]


def load_dataset_discussions():
    all_disc_data = []
    
    discussions = read_file_lines(path_data,f1)
      
    for lin in discussions:
        disc_data = []
        
        dados = lin.split(";")
        id_discussion = dados[0]
        category_discussion = dados[1]
        
        if (category_discussion == category):
        #if (True): #For category = "ALL"
            disc_data.append(str(id_discussion))
            
            title_discussion = " ".join(dados[5:])
            main_discussion_text = read_file_content(path_data+"discussions/"+dir_disc_main_post+"/",id_discussion+".txt")
            
            disc_text = title_discussion + " " + main_discussion_text   
            
            text_clean = pre_process(disc_text)
        
            # Detecting zero-length discussion posts
            #if (len(text_clean) == 0):
             #print(id_discussion,";",main_discussion_text) 
    
            disc_data.append(str(disc_text))
            disc_data.append(str(text_clean))
            all_disc_data.append(disc_data)
                   
        
    return(all_disc_data)


all_disc_data = []
all_disc_data = load_dataset_discussions()
print("#Discussion posts:",len(all_disc_data))



In [ ]:
discussions_df = pd.DataFrame (all_disc_data, columns = ['ident', 'content','cleaned'])
print (discussions_df)

### RD-Detector: Relatedness Checker

#### Relatedness Checker: Similarity Checher

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
#loading sentence-Bert model:all-mpnet-base-v2
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
#ceating the discussions' embeddings
document_embeddings = sbert_model.encode(discussions_df['cleaned'], convert_to_tensor=True)

In [ ]:
#defining the K value
#doc: https://www.sbert.net/examples/applications/paraphrase-mining/README.html
#top_k – For each sentence, we retrieve up to top_k other sentences
#score_function – Function for computing scores. By default, cosine similarity.

k= 2 #k=5
paraphrases = util.paraphrase_mining(sbert_model, discussions_df['cleaned'], corpus_chunk_size=len(discussions_df['cleaned']), top_k = k)

#### Relatedness Checker: Threshold definition (T_related)  & Selection of the related discussion candidates

In [ ]:
def select_related_discussion_candidates(sim_values,l_inf,N):
    tot_disc=0;
    similares = []
    
    for master, target, sim1 in sim_values:
        sim = float(sim1)

        if (sim >= l_inf):
            #selecting unique pairs of related discussion candidates
            if (target+";"+master not in similares):               
                r_candidates = master + ";" + target
                similares.append(r_candidates)
                tot_disc +=1                
                #saving the RD-Detector output
                save_file(path_data+"rd-detector_output/",r_candidates,"R_"+project_name+"top_"+str(N)+"_"+category+".csv")                                         
    return (tot_disc)

In [ ]:
all_sim_top=[]
all_bert_values =[]

for paraphrase in paraphrases:
    score, i, j = paraphrase  
    #(master,target,similarity_value)
    tupla = (discussions_df.iloc[i]['ident'],discussions_df.iloc[j]['ident'], score)
    all_bert_values.append(tupla)
    all_sim_top.append(score)

#saving the similarity values    
save_tupla(all_bert_values,project_name+"_top"+str(k)+"_"+category+"_similaryValues.csv","")


# descriptive statistic
v_all_sim = np.array(all_sim_top)
q75, q50, q25 = np.percentile(v_all_sim, [75 , 50, 25])
iqr = q75 - q25 

# threshold t_related == innerfence
innerfence = q75 + (1.5*iqr)
    
print("******")
print("Top:",k)
print("Q25:",q25)
print("Median:",q50)
print("Q75:",q75)
print("IQR:",iqr)
print("Innerfence value:",innerfence)

### OUTPUT - Selection of the related discussion candidates
select_related_discussion_candidates(all_bert_values,innerfence,k)